## First Datasets Analisys DWM 2021 Project

TO DO:
- Add custom features to your dataset (e.g., avg. prices in the same geographical
area) and evaluate their contribution
- Compare at least two prediction methods, after fine-tuning their parameters
- Investigate instances with the most correct predictions and with the most wrong
predictions. Do they have some peculiarities? Any strange feature distribution?
- To get a good mark: more prediction methods, not trashing features, external
data, other ML Libraries, prediction methods not discussed during the course, ecc

Durante l'orale posso mostrare dei dati che non sono nella relazione di 5 pagine consegnata (es dei grafici particolari)

Prediction Task: Home Value Prediction: https://www.kaggle.com/c/zillow-prize-1/overview

In [ ]:
import pandas as pd
import numpy as np

input_folder='data/'
data_file_name='properties_2017.parquet'
#feature_dict_file_name='zillow_data_dictionary.xlsx'

In [ ]:
#Convert file from CSV to Parquet
'''df_houses=pd.read_csv(input_folder+properties_2017.csv, header=0)
df_houses.to_parquet(input_folder+"properties_2017.parquet")''';

In [ ]:
df_train=pd.read_csv(input_folder+"train_2017.csv")
df_train.info()
print('Shape: ', df_train.shape)

In [ ]:
print(df_train['parcelid'].duplicated().sum())
print(df_train[['parcelid', 'transactiondate']].duplicated().sum()) #stessa casa venduta in date diverse con prezzi diversi
#mi serve una feature/features che catturi questa diversità in base alla data (differenza di qualcosa, correlazioni ecc), oppure introdurre la data come features
#es se la ho: data di costruzione della casa posso relazionare la "vecchiaia" in base alla data di vendita e il prezzo

In [ ]:
#feature_dict=pd.read_excel(input_folder+feature_dict_file_name)
df_houses=pd.read_parquet(input_folder+data_file_name)
df_houses[df_houses.select_dtypes(np.float64).columns] = df_houses.select_dtypes(np.float64).astype(np.float32)
#risparmio circa 600mb di memoria e probabilmente alcune operazioni sono più veloci 

df_houses.info()
print('Shape: ', df_houses.shape)
#può essere utile salvare la variabile con %store o salvando in un file

In [ ]:
df_houses.head(10)

In [31]:
df_houses.select_dtypes(exclude=[np.float32]).columns
#possible categorical column


Index(['parcelid', 'hashottuborspa', 'propertycountylandusecode',
       'propertyzoningdesc', 'fireplaceflag', 'taxdelinquencyflag'],
      dtype='object')

In [ ]:
#df_houses.isna().sum()
#NAN percentage:
df_houses.isna().sum()/df_houses.shape[0] *100
#se ci sono tanti tanti nan e la colonna non la considero importante la posso droppare
#o posso rimpiazzare il valore e mettere una colonna con un flag se rimpiazzato o no, o usare random forest
#per la colonne categoriali posso usare one hot encoding e se ho una categoria "other" posso aggregare insieme quella con il missing

In [ ]:
#cerco di selezionare un dataset con pochi missing alleno e tutto fino alla fine e poi lo raffino
#utile anche se uso random forest per "iniziare" le sostituzioni

In [ ]:
#df_houses['parcelid'].duplicated().sum() è univoco

Ma se una istanza (riga) ha molti nan?
Se nel train posso dropparla, nel test no

Se ho tante categorie? Tipo la via?
- Posso aggregare per macrocategorie (es per regione) così diminuisco
- Oppure scelgo le 10 più frequenti ad esempio e le altre le metto a "other"
- per ottimizzare se ho una variabile ordinale (alto, medio, basso), la posso codificare come interi ordinati ugualmente

Posso provare a destrutturare il dataset (ad esempio per regionidcountry) ed allenare 3 modelli per i 3 cluster trovati in base alla regione

Provo a vedere qualcosa con PCA (Principal
 Component Analisys) con 2/3 componenti per disegnare con un grafico le features principali che contrubuiscono con la prima o la seconda componente (vedo come fatto a lezione)

PERFORMANCE: Posso provare a campionare solo una parte del dataset e non tutto per efficienza di memoria (metà dataset, 1/4) e decidere il modello e in tuning. Una volta finito questo se basta la memoria provo ad allenarlo su tutto. Posso provare ad usare il notebook di Colab o di Kaggle

ENSAMBLE DI ALGORITMI: https://www.kaggle.com/code/abhisheksharma26jan/zillowabhishek
(nel video del 06/12 in realtà parla di un altro notebook)

ENSAMBLE STACKING
Modello che fa la media (o media pesata) dei modelli passati diversi.
Il modello è fatto da istanze (case id) sulle righe e nome dell'alg sulle y, nelle celle c'è il valore grezzo predetto.
La media pesata, cioè il valore dei pesi tra tutti è una regressione lineare, trovare la combinazione ottima di pesi dei vari modelli

ENSAMBLE CASCADING
Per ogni istanza uso un modello per decidere la strada da prendere e passare loutput un altro modello a cascata basato sulla strada scelta da quello prima. Es se ho una features costosa da calcolare creo un modello senza questa features, se per un istanza questo modello mi dice che ha senso calcolare la features gli passo l'istanza a sto modello con la features costosa e la calcolo, altrimenti no (uso un filtro)